# News Article Clustering

In this track you're asked to cluster news articles into the topics. You will use a subset of [fetch_20newsgroups](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) dataset. 

## Marking criteria: 

- **8 points**: Implement TF-IDF for words vectorization.
- **2 points**: Reduce dimensionality with TruncatedSVD and answer the theoretical questions about it.
- **10 points**: Implement DBSCAN clustering algorithm.

## Baseline 

You can look at the baseline, or the basic solution of this task. Your assignments will be to implement TF-IDF vectorized and DBSCAN from scratch and append dimension reduction on the token vectors to improve the results.

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import DBSCAN
import pandas as pd

# Data set
# Take only 3 categories from 20
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'),
                                      categories=['alt.atheism', 'talk.religion.misc', 'comp.graphics'])
# Example of the sample
print(f"Sample:\n{newsgroups_train.data[0]}\n")

# Tokenization the articles
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english')
X = vectorizer.fit_transform(newsgroups_train.data)

# Convert TF-IDF to pd.Dataframe for simplicity
tfidf_tokens = vectorizer.get_feature_names_out()
df_tfidfvect = pd.DataFrame(data=X.toarray(), columns=tfidf_tokens)
print(f"TF-IDF table:\n {df_tfidfvect}\n")

# Reduce the dimension
svdT = TruncatedSVD(n_components=50)
svdTFit = svdT.fit_transform(df_tfidfvect)

# Clusterization
db = DBSCAN(eps=0.5, min_samples=5).fit(svdTFit)
skl_labels = db.labels_

# See how many clusters we've got (ideally, should be 3 and some outliers marked as -1)
skl_labels.sort()
print(f"Article labels {skl_labels}")

Sample:
Those things,
	which ye have both learned, and received,
	and heard, and seen in me,
	do:
	and the God of peace shall be with you.

TF-IDF table:
        00  000   01  0184   02  0200   03   04   05  0511  ...  zlumber  zone   
0     0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0   0.0  ...      0.0   0.0  \
1     0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0   0.0  ...      0.0   0.0   
2     0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0   0.0  ...      0.0   0.0   
3     0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0   0.0  ...      0.0   0.0   
4     0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0   0.0  ...      0.0   0.0   
...   ...  ...  ...   ...  ...   ...  ...  ...  ...   ...  ...      ...   ...   
1436  0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0   0.0  ...      0.0   0.0   
1437  0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0   0.0  ...      0.0   0.0   
1438  0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0   0.0  ...      0.0   0.0   
1439  0.0  0.0  0.0   0.0  0.0   0.

### 1 Words vectorization & Dimensionality reduction by TruncatedSVD [10 points]

As far as you have short articles (or set of words, or **documents**) as data points, you cannot apply clusterization to them directly. Thus, you need to encode the articles somehow. In this assignment you're suggested to use TF-IDF as a tokenizer. TF-IDF (Term Frequency - Inverse Document Frequency) get the corpus of sentences and produce a matrix $T$ with shape $N$ x $M$, where $N$ - number of sentences, $M$ - number of **all** unique words in the whole corpus. The item of the matrix $T_i,j$ ($i-th$ word in $j-th$ document) is calculated as follows:

$$T(i,j) = tf(i,j) * idf(i) $$

where

$$tf(i, j) = {n_{i,j} \over \sum_j n_{k,j}}$$

$n_{i, j} $ - number of entrances of the word in the sentence,${\sum_j n_ {k, j}} $- overall number of words in the document. So, it's a simple frequency of a particular word in a sentence; note it can be equal to zero if the word does not appear in the document. $idf(i) $ is equal to:

$$idf(i) = log {|D| \over |{d_k \in D, | n_i \in d_k}|}$$ 

$|D|$ - overall number of documents, and the denominator represents the number of documents, containing the word $n_i$.

By the end of the day, all of your articles are encoded into the TF-IDF matrix as its row, have the same dimensionality and might be used for clustering algorithms. 

While we do not work with semantic analysis of the text for clusterization, such tokens would implicitly encode the structure of the documents: we can suppose that the documents from one category (e.g. sports) utilize the similar words. TF-IDF gains with higher value the words that appear a lot in the several documents, but not in all corpus. Also note that the TF-IDF matrix would be very sparse, and you'll need to reduce its dimensionality.

### 1.1 TF-IDF [8 points]

In this task you're asked to **implement the formulas above** from scratch to calculate a TF-IDF matrix. 

You also have a subtask, which is important from the practical view: data cleaning. The articles contain a lot of punctuation, special characters and various forms of the same words that you need to handle. For example, words *clusters*, *Clusters*, *clusters? * *_clusters* without preprocessing would be considered as different, which greatly increase the number of columns in a TF-IDF matrix and reduce its descriptive power. Therefore, **before** the calculation of the metrics, it's recommended to:
1) Remove all punctuation marks;
2) Remove special symbols, such as `\n` and `\t`;
3) Remove stop words, i.e. the most common and non-informative, such a `a`, `the`, `I`, etc. You can use `nltk` package to load them;
4) Convert every word in the lower case.

Also, before the calculation you can remove the words that are too rare in the corpus, for example, appear less than 2 times (see [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) documentation). 

***Do not use any additional libraries***

In [8]:
import string
import math
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
stops = set(stopwords.words("English"))

#  load 3 subsets in the same manner as in the baseline
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'),
                                      categories=['alt.atheism', 'talk.religion.misc', 'comp.graphics'])


def preprocess_text(text):
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = text.replace("\n", "").replace("\t", "")
    text = text.lower()
    words = word_tokenize(text)
    words = [word for word in words if word not in stops]
    preprocessed_text = " ".join(words)

    return preprocessed_text


preprocessed_documents = [preprocess_text(document) for document in newsgroups_train.data]

vocabulary = set()
for document in preprocessed_documents:
    words = document.split()
    vocabulary.update(words)

word_frequencies = {}
for document in preprocessed_documents:
    words = document.split()
    for word in words:
        if word not in word_frequencies:
            word_frequencies[word] = 0
        word_frequencies[word] += 1

filtered_vocabulary = [word for word in vocabulary if word_frequencies[word] >= 2]

num_documents = len(preprocessed_documents)

tf_idf_matrix = np.zeros((num_documents, len(filtered_vocabulary)))
for i, doc in enumerate(preprocessed_documents):
    words = doc.split()
    for j, word in enumerate(filtered_vocabulary):
        tf = words.count(word) / len(words) if len(words) > 0 else 0
        idf = math.log(num_documents / word_frequencies[word])
        tf_idf = tf * idf
        tf_idf_matrix[i, j] = tf_idf

# pandas used only for final dataframe formation
tf_idf_df = pd.DataFrame(tf_idf_matrix, columns=list(filtered_vocabulary))
tf_idf_df

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,traditionally,withyou,notwhy,interaction,magian,authored,irix,peculiar,affecting,evidencefor,...,qp,womans,conclusively,approximation,andfalling,isso,reinforcing,theimage,problemwith,entrails
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 1.2 TruncatedSVD [2 points]

Since the number of features of the resulting frame is pretty big, it would not be reasonable to clusterize them directly. However, PCA algorithm is not applicable here, because the data is too sparse (try yourself, sklearn should warn you!). Instead of that we are going to use **Truncated Singular Value Decomposition**.

You can use the realization of TruncatedSVD from sklearn. You're asked to read the following articles and pages [1](https://en.wikipedia.org/wiki/Singular_value_decomposition), [2](https://langvillea.people.cofc.edu/DISSECTION-LAB/Emmie%27sLSI-SVDModule/p5module.html) and answer the questions:

1) What are the ways to choose the proper number of eigenvalues in truncated SVD?
    - Plotting singular values in decreasing order and examining point of diminishing returns can help determine number of significant eigenvalues to retain. The elbow represents drop-off point where eigenvalues become less significant.
    - Calculating cumulative sum of explained variances of eigenvalues and choosing number of eigenvalues that explain a desired percentage of total variance.
    - Having prior knowledge about dataset or specific problem at hand can guide the selection of number of eigenvalues. For example, if dataset is related to images and objective is to capture distinct visual features then domain knowledge can help in determining number of eigenvalues that represent those features.
2) In which cases truncated SVD is not appropriate?
    - Truncated SVD is specifically designed for sparse or high-dimensional data. If data is not sparse and has a low-dimensional representation traditional principal component analysis can be more suitable.
    - Truncated SVD assumes that data is linearly independent. If data exhibits a strong linear dependency truncated SVD may not provide meaningful results. In such cases other techniques such as non-linear dimensionality reduction methods might be more appropriate.
3) Are there any specific preprocessing steps that should be taken before applying truncated singular value decomposition to a raw text corpus?
    - Tokenization
    - Stop word removal
    - Cleaning and normalization
    - Term frequency-inverse document frequency weighting
    - Sparse matrix representation

In [9]:
from sklearn.decomposition import TruncatedSVD

my_data = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

svdT = TruncatedSVD(n_components=2)
svdTFit = svdT.fit_transform(my_data)

### 2. DBSCAN [10 points]

Now everything is ready for clusterization. In this task you're asked to **implement** DBSCAN algorithm. We do not provide any guidelines, but you can use the lecture slides.

The data set is not the easiest one for clusterization, even with such a powerful algorithm. Thus, your goal is to realize DBSCAN that divides it to at least **two clusters**, but **not more than six**, except the outliers. Print the labels as a proof. In case you have done everything you could, but still have only one cluster, choose other 3 topic from fetch_20newsgroups ([check](https://scikit-learn.org/stable/datasets/real_world.html#newsgroups-dataset) for target names). The distinct articles can be more separable. 

***Do not use any additional libraries***

In [10]:
from sklearn import metrics
import numpy as np
from math import sqrt


def euclidean_distance(point1, point2):
    squared_distance = np.sum(np.square(point1 - point2))

    return sqrt(squared_distance)


def get_neighbors(data, point_index, epsilon):
    neighbors = []
    for i, point in enumerate(data):
        if euclidean_distance(data[point_index], point) <= epsilon:
            neighbors.append(i)

    return neighbors


def dbscan(data, epsilon, min_points):
    labels = np.zeros(len(data), dtype=int)
    cluster_id = 1
    for i, point in enumerate(data):
        if labels[i] != 0:
            continue
        neighbors = get_neighbors(data, i, epsilon)
        if len(neighbors) < min_points:
            labels[i] = -1
        else:
            labels[i] = cluster_id
            while len(neighbors) > 0:
                current_point = neighbors[0]
                if labels[current_point] == -1:
                    labels[current_point] = cluster_id
                elif labels[current_point] == 0:
                    labels[current_point] = cluster_id
                    current_point_neighbors = get_neighbors(data, current_point, epsilon)
                    if len(current_point_neighbors) >= min_points:
                        neighbors.extend(current_point_neighbors)
                neighbors = neighbors[1:]
            cluster_id += 1

    return labels

In [11]:
data = np.array([[1, 2], [2, 3], [3, 2], [8, 7], [7, 8], [8, 9]])
epsilon = 2.0
min_points = 2

labels = dbscan(data, epsilon, min_points)
print("Cluster Labels:", labels)

silhouette_score = metrics.silhouette_score(data, labels)
print("Silhouette Score:", silhouette_score)


Cluster Labels: [1 1 1 2 2 2]
Silhouette Score: 0.8000313669777999


### References

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html
https://en.wikipedia.org/wiki/Tf–idf